In [ ]:
import simplejson

import numpy as np
import xarray as xr
import geopandas as gpd

import matplotlib.pyplot as plt

In [ ]:
regions = gpd.read_file("../data/regions.geojson")

with open("../data/metadata.json") as f:
    metadata = simplejson.load(f)


def export(name, out):
    with open(f"../data/{name}.json", "w") as f:
        simplejson.dump(out, f, sort_keys=True, ignore_nan=True)

In [ ]:
ds = xr.open_dataset("models-dist/Tas_percentiles_final.nc")
ds

In [ ]:
cdf = {}

for _, region in regions.iterrows():
    nuts_id = region["id"]

    if nuts_id not in ds["region"]:
        cdf[nuts_id] = None
        continue

    cdf[nuts_id] = []
    for model in metadata["models"]:
        model_id = f"{model["gcm"]}*{model["rcm"]}"

        if model_id not in ds["model"]:
            cdf[nuts_id].append(None)
            continue

        percentiles = []
        for p in [0, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99, 100]:
            if p in ds["percentile"]:
                percentiles.append(
                    ds["temperature_percentile"].sel({
                        "model": model_id,
                        "region": nuts_id,
                        "percentile": p
                    })
                    .values
                    .astype(np.float64)
                    .round(2)
                )
            else:
                percentiles.append(None)
        cdf[nuts_id].append(percentiles)

In [ ]:
export("cdfs", cdf)
